<a href="https://colab.research.google.com/github/madhu-sv/asl-sign-language-recognition/blob/master/ASLDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASL Sign Language Recognition

This notebook demonstrates how to train and evaluate a Convolutional Neural Network (CNN) model to recognize American Sign Language (ASL) signs using TensorFlow and Keras.

## Step 1: Install Required Libraries

First, we need to install the necessary libraries.


In [1]:
# Install required libraries
!pip install tensorflow
!pip install matplotlib
!pip install opencv-python

##Step 2: Import Libraries
Next, import the necessary libraries for building and training the model.

In [ ]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2


##Step 3: Mount Google Drive
Mount Google Drive to access the dataset stored in your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


##Step 4: Data Preprocessing
Data Augmentation and Preprocessing Functions
Define functions to load and preprocess the dataset, including data augmentation.

In [ ]:
def parse_image(filename, label, num_classes):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [64, 64])
    img = img / 255.0
    label = tf.one_hot(label, num_classes)
    return img, label

def load_dataset(data_dir, batch_size=32, augment=False):
    classes = sorted(os.listdir(data_dir))
    class_indices = {cls: idx for idx, cls in enumerate(classes)}

    filepaths = []
    labels = []

    for cls in classes:
        class_dir = os.path.join(data_dir, cls)
        for img in os.listdir(class_dir):
            filepaths.append(os.path.join(class_dir, img))
            labels.append(class_indices[cls])

    filepaths = tf.constant(filepaths)
    labels = tf.constant(labels)

    num_classes = len(class_indices)

    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    dataset = dataset.map(lambda x, y: parse_image(x, y, num_classes), num_parallel_calls=tf.data.experimental.AUTOTUNE)

    if augment:
        dataset = dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = dataset.map(lambda x, y: (tf.image.random_brightness(x, max_delta=0.2), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = dataset.map(lambda x, y: (tf.image.random_contrast(x, lower=0.8, upper=1.2), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)

    dataset = dataset.shuffle(buffer_size=len(filepaths))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset, num_classes

def create_data_generators(train_dir=None, test_dir=None, batch_size=32):
    train_dataset, test_dataset = None, None
    num_classes = 0

    if train_dir:
        train_dataset, num_classes = load_dataset(train_dir, batch_size=batch_size, augment=True)

    if test_dir:
        test_dataset, num_classes = load_dataset(test_dir, batch_size=batch_size, augment=False)

    return train_dataset, test_dataset, num_classes


##Step 5: Build the CNN Model
Define the CNN model architecture with regularization techniques to prevent overfitting.

In [ ]:
def build_model(input_shape=(64, 64, 3), num_classes=29):
    """
    Build a Convolutional Neural Network model.

    Args:
        input_shape (tuple): Shape of the input images.
        num_classes (int): Number of output classes.

    Returns:
        model: A compiled Keras model.
    """
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


##Step 6: Train the Model
Define the training function and use callbacks for early stopping and model checkpointing.

In [ ]:
def train_model(train_dir, test_dir, epochs=50, batch_size=32):
    """
    Train the CNN model on the ASL dataset using tf.data.

    Args:
        train_dir (str): Path to the training data directory.
        test_dir (str): Path to the test data directory.
        epochs (int): Number of epochs to train the model.
        batch_size (int): Batch size for training.
    """
    train_dataset, test_dataset, num_classes = create_data_generators(train_dir, test_dir, batch_size=batch_size)

    print(f"Number of classes: {num_classes}")

    model = build_model(num_classes=num_classes)

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_asl_model.keras', save_best_only=True)

    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=test_dataset,
        callbacks=[early_stopping, model_checkpoint]
    )

    model.save('asl_model.keras')

    test_loss, test_acc = model.evaluate(test_dataset)
    print(f'Test accuracy: {test_acc}')
    return history

# Define paths to training and test data
train_dir = '/content/drive/MyDrive/asl_data/train'
test_dir = '/content/drive/MyDrive/asl_data/test'

# Train the model
history = train_model(train_dir, test_dir)


##Step 7: Evaluate the Model
Evaluate the trained model on the test dataset to determine its performance.

In [ ]:
def evaluate_model(test_dir):
    """
    Evaluate the trained CNN model on the ASL test dataset.

    Args:
        test_dir (str): Path to the test data directory.
    """
    _, test_dataset, num_classes = create_data_generators(test_dir=test_dir, batch_size=32)
    model = tf.keras.models.load_model('best_asl_model.keras')

    test_loss, test_acc = model.evaluate(test_dataset)
    print(f'Test accuracy: {test_acc}')
    return test_loss, test_acc

# Evaluate the model
evaluate_model(test_dir)


##Step 8: Visualize Training History
Visualize the training and validation accuracy and loss over epochs.

In [ ]:
def plot_training_history(history):
    """
    Plot the training history.

    Args:
        history: History object from model training.
    """
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.show()

# Plot training history
plot_training_history(history)
